In [1]:
from italian_csv_type_prediction.dataframe_generators import SimpleDatasetGenerator
from italian_csv_type_prediction.models import TypePredictor
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [2]:
def dataset_generation(number:int):
    return SimpleDatasetGenerator().build(number, verbose=False)

def _dataset_generation(args):
    return dataset_generation(*args)

def parallel_dataset_generation(number:int):
    processes = min(cpu_count()*5, number)
    with Pool(cpu_count()) as p:
        Xs, ys = list(zip(*tqdm(
            p.imap(_dataset_generation, (
                (number//processes, )
                for _ in range(processes)
            )),
            desc="Creating dataset",
            total=processes,
            leave=False
        )))
    return np.vstack(Xs), np.concatenate(ys)

In [3]:
x_train, y_train = parallel_dataset_generation(1000)
x_test, y_test = parallel_dataset_generation(1000)

In [4]:
model = TypePredictor()

model.fit(x_train, y_train)

In [5]:
y_pred = model._model.predict(x_test)
y_train_pred = model._model.predict(x_train)

In [6]:
accuracy_score(y_test, y_pred), balanced_accuracy_score(y_test, y_pred)

(0.9971315872414529, 0.9967686476752264)

In [7]:
accuracy_score(y_train, y_train_pred), balanced_accuracy_score(y_train, y_train_pred)

(0.9995393820438141, 0.9995224610374838)

In [8]:
X, y = SimpleDatasetGenerator().generate_simple_dataframe(max_rows=10)

In [9]:
y

,ItalianFiscalCode,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,Year,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
0,ItalianVAT,CadastreCode,Document,NaN,NaN,ItalianZIPCode,ProvinceCode,Region,NaN,Year,...,CountryCode,Name,NaN,String,NaN,PhoneNumber,Date,BiologicalSex,Error,NaN
1,ItalianFiscalCode,CadastreCode,Document,Plate,Address,ItalianZIPCode,NaN,Region,Municipality,NaN,...,CountryCode,NaN,Surname,NaN,EMail,NaN,NaN,BiologicalSex,Boolean,NumericId
2,ItalianFiscalCode,NaN,Document,NaN,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,Year,...,CountryCode,Name,Surname,NaN,EMail,NaN,NaN,BiologicalSex,Boolean,NumericId
3,ItalianFiscalCode,CadastreCode,Document,Plate,NaN,Error,ProvinceCode,NaN,Municipality,Year,...,CountryCode,Name,Surname,String,NaN,PhoneNumber,Date,BiologicalSex,Boolean,NaN
4,NaN,CadastreCode,Document,NaN,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,Year,...,NaN,Name,NaN,NaN,EMail,PhoneNumber,Date,BiologicalSex,NaN,NumericId
5,ItalianVAT,Error,Document,Plate,NaN,Error,ProvinceCode,Region,Municipality,Error,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
6,ItalianFiscalCode,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,NaN,Year,...,CountryCode,Name,Surname,NaN,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
7,ItalianFiscalCode,CadastreCode,Document,NaN,Address,NaN,ProvinceCode,Region,Municipality,NaN,...,CountryCode,Name,Surname,NaN,NaN,NaN,Date,BiologicalSex,Boolean,NaN
8,ItalianVAT,NaN,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Error,Year,...,CountryCode,NaN,NaN,String,NaN,PhoneNumber,Date,BiologicalSex,Boolean,NumericId


In [10]:
model.predict_dataframe(X)

,ItalianFiscalCode,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,Year,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
0,ItalianVAT,CadastreCode,Document,NaN,NaN,ItalianZIPCode,ProvinceCode,Region,NaN,Year,...,CountryCode,Name,NaN,Document,NaN,PhoneNumber,Date,BiologicalSex,Error,NaN
1,ItalianFiscalCode,CadastreCode,Document,Plate,Address,ItalianZIPCode,NaN,Region,Municipality,NaN,...,CountryCode,NaN,Surname,NaN,EMail,NaN,NaN,BiologicalSex,Boolean,NumericId
2,ItalianFiscalCode,NaN,Document,NaN,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,Year,...,CountryCode,Name,Surname,NaN,EMail,NaN,NaN,BiologicalSex,Boolean,NumericId
3,ItalianFiscalCode,CadastreCode,Document,Plate,NaN,Error,ProvinceCode,NaN,Municipality,Year,...,CountryCode,Name,Surname,Document,NaN,PhoneNumber,Date,BiologicalSex,Boolean,NaN
4,NaN,CadastreCode,Document,NaN,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,Year,...,NaN,Name,NaN,NaN,EMail,PhoneNumber,Date,BiologicalSex,NaN,NumericId
5,ItalianVAT,Error,Document,Plate,NaN,Error,ProvinceCode,Region,Municipality,Error,...,CountryCode,Name,Surname,Document,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
6,ItalianFiscalCode,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,NaN,Year,...,CountryCode,Name,Surname,NaN,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
7,ItalianFiscalCode,CadastreCode,Document,NaN,Address,NaN,ProvinceCode,Region,Municipality,NaN,...,CountryCode,Name,Surname,NaN,NaN,NaN,Date,BiologicalSex,Boolean,NaN
8,ItalianVAT,NaN,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Error,Year,...,CountryCode,NaN,NaN,Error,NaN,PhoneNumber,Date,BiologicalSex,Boolean,NumericId


In [11]:
from collections import Counter

mask = y_test != y_pred

true_labels = model._embedder._encoder.inverse_transform(y_test[mask])
predicted_labels = model._embedder._encoder.inverse_transform(y_pred[mask])

Counter(zip(true_labels, predicted_labels))

Counter({('NaN', 'CountryCode'): 96,
         ('Error', 'BiologicalSex'): 7,
         ('Error', 'NumericId'): 53,
         ('Document', 'Name'): 3,
         ('Error', 'Name'): 20,
         ('Error', 'ProvinceCode'): 23,
         ('ItalianZIPCode', 'Error'): 15,
         ('CountryCode', 'NaN'): 79,
         ('Error', 'Document'): 143,
         ('String', 'Error'): 143,
         ('BiologicalSex', 'Error'): 20,
         ('ItalianVAT', 'Error'): 11,
         ('ItalianFiscalCode', 'Error'): 14,
         ('Error', 'Region'): 4,
         ('EMail', 'Error'): 20,
         ('Error', 'String'): 14,
         ('Error', 'CountryCode'): 23,
         ('ItalianZIPCode', 'NumericId'): 273,
         ('Document', 'Error'): 58,
         ('Error', 'ItalianVAT'): 23,
         ('Error', 'ItalianZIPCode'): 5,
         ('Integer', 'Error'): 30,
         ('Integer', 'NumericId'): 5,
         ('NumericId', 'ItalianZIPCode'): 402,
         ('Name', 'Error'): 59,
         ('Error', 'ItalianFiscalCode'): 21,
       